<a href="https://colab.research.google.com/github/tcglarry/kg_power_line/blob/master/vsb_power_line_first_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -*- coding: utf-8 -*-
"""vsb_power_line_eda.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1AZHJmuSwKCP8IOrOg_J96XmoQXDH-O-w
"""

# Step 1
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110842 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [3]:

# Step 2
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [0]:
src= 'drive/My Drive/kaggle/vsb_power_line/data/'
src_train= 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data/train/'

src_valid= 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data/validation/'

src_test = 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data/test_data/'

In [5]:
!ls 'drive/My Drive/kaggle/vsb_power_line/data/'

all.zip		       train_test_data
id_label.csv	       train_test_data_2
metadata_test.csv      vsb_power_line_DenseNet121_1st__acc_2.h5
metadata_train.csv     vsb_power_line_DenseNet121_1st__acc.h5
new_train.csv	       vsb_power_line_DenseNet121_1st__loss_2.h5
sample_submission.csv  vsb_power_line_DenseNet121_1st__loss.h5
test.parquet	       vsb_power_line_eda.ipynb
train.parquet


In [37]:
# -*- coding: utf-8 -*-


#==============================================================================
# # # Module
#==============================================================================

from __future__ import print_function


import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import subprocess
import os
import pickle
import sklearn 

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report


from xgboost.sklearn import XGBClassifier




import keras.backend as K

from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121, DenseNet169,DenseNet201, preprocess_input
#from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenetv2 import MobileNetV2
from keras.applications.nasnet import NASNetMobile,NASNetLarge

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,  img_to_array, load_img



from keras.applications.xception import Xception, preprocess_input

from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, \
Reshape, Dropout,Cropping2D,Activation, BatchNormalization
from keras.models import Model, load_model
import pandas as pd

from keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras.callbacks import Callback

from keras.optimizers import Adam

from sklearn.metrics import accuracy_score
from keras import regularizers
#from dummyPy import OneHotEncoder
from keras.utils import to_categorical

from sklearn.metrics import confusion_matrix

import itertools
import time

import gc



ImportError: ignored

In [0]:

train_read_path = src_train ##
val_read_path = src_valid ##

h_size,w_size= 144,288
size, channel = (144,288), 3

num_classes = 2


batch = 64


save_dir =  src
save_file = 'vsb_power_line_DenseNet121_1st_'


In [11]:

#==============================================================================
# # # Train - Generator
#==============================================================================
'''
train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=[0.9, 1.1], 
        fill_mode = 'reflect')
'''

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    shear_range=0.2,
    width_shift_range=1.0,
    height_shift_range=1.0,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(
    directory=src_train,
    target_size=(h_size,w_size),
    color_mode="rgb",
    batch_size=batch,
    class_mode="binary",
    shuffle=True,
    seed=42
)


valid_datagen = ImageDataGenerator(
        rescale=1./255)




valid_generator = valid_datagen.flow_from_directory(
    directory=src_valid,
    target_size=(h_size,w_size),
    color_mode="rgb",
    batch_size=batch,
    class_mode="binary",
    shuffle=False,
    seed=42
)


Found 6970 images belonging to 2 classes.
Found 1742 images belonging to 2 classes.


In [12]:
train_generator.classes

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [0]:
class_weight = {0:1,1:17}

In [0]:


def build_model_2():
    inputs = Input(shape=(h_size,w_size,3))
    #inputs = Lambda(lambda x: x/275. - 1.0)(inputs)
    # create the base pre-trained model
    base_model =DenseNet121(weights='imagenet',input_tensor=inputs,  include_top=True)



    x = base_model.layers[51].output
    print ('shape is = ', x.get_shape())

    x= GlobalAveragePooling2D()(x)
    print ('shape is = ', x.get_shape())
    

    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = Dense(64,activation = 'sigmoid')(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    
    x = Dropout(0.2)(x)
    x = Dense(32,activation = 'sigmoid')(x)

    
    '''
    
    x = Dense(128,activation='relu')(x)
    
    x= BatchNormalization()(x)
    x= Activation('relu')(x)

    #x = Dropout(0.3)(x)
    #x = Dense(32,activation='relu')(x)
    
    x = Dense(32,activation='linear', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    
    
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    '''
    
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = Dropout(0.2)(x)   #BN with dropout ?? try .
    
    output = Dense(1,activation = 'sigmoid')(x)


    model = Model(inputs=base_model.input, outputs=output)
    model.summary()

    return model

In [15]:

#==============================================================================
# # # Train - Fit - ResNet
#==============================================================================

model = build_model_2()

#model = load_model(save_dir+'Stage_1_DenseNet121_binary_generator_1_acc.h5')





33193984/33188688 [==============================] - 1s 0us/step
shape is =  (?, 36, 72, 128)
shape is =  (?, 128)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 144, 288, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 150, 294, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 72, 144, 64)  9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 72, 144, 64)  256         conv1/conv[0][0]            

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
def lr_epoch(epochs):
    learning_rate_ = 0.0102
    decay= 0.01/100.
    lr = learning_rate_ - epochs*decay
    return lr

In [20]:


Step_Train = train_generator.n//train_generator.batch_size

print (train_generator.n, train_generator.batch_size)
Step_Valid = valid_generator.n

#define callback


#define callback
model_save_loss = save_dir+save_file+'_loss_2.h5'
model_save_acc = save_dir+save_file+'_acc_2.h5'
checkpoint_loss = ModelCheckpoint(model_save_loss, monitor='val_loss',save_best_only=True, mode='auto')
checkpoint_acc = ModelCheckpoint(model_save_acc, monitor='val_acc',save_best_only=True, mode='auto')
earlystop = EarlyStopping(monitor='val_loss',  patience=20,  mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=10, min_lr=0.00001,verbose=1)
#（best val accuarcy saved 
lr_scheduler= LearningRateScheduler(lr_epoch,verbose=1)
#callback_list = [checkpoint_loss,checkpoint_acc, earlystop,lr_scheduler]







6970 64


In [21]:
callback_list_1 = [checkpoint_loss,checkpoint_acc, earlystop]


# starts trauning 
his = model.fit_generator(generator=train_generator,
                            steps_per_epoch=Step_Train,
                            validation_data=valid_generator,
                            validation_steps=Step_Valid,
                            epochs=150, verbose = 1,
                             callbacks= callback_list_1, 
                            class_weight=class_weight)

Epoch 1/150
108/108 [==============================] - 2641s 24s/step - loss: 0.5601 - acc: 0.7248 - val_loss: 0.2476 - val_acc: 0.9399
Epoch 2/150
108/108 [==============================] - 807s 7s/step - loss: 0.2741 - acc: 0.9320 - val_loss: 0.2359 - val_acc: 0.9399
Epoch 3/150
108/108 [==============================] - 809s 7s/step - loss: 0.2282 - acc: 0.9393 - val_loss: 0.2318 - val_acc: 0.9399
Epoch 4/150
108/108 [==============================] - 807s 7s/step - loss: 0.2150 - acc: 0.9385 - val_loss: 0.2321 - val_acc: 0.9399
Epoch 5/150
108/108 [==============================] - 821s 8s/step - loss: 0.2060 - acc: 0.9402 - val_loss: 0.2165 - val_acc: 0.9389
Epoch 6/150
107/108 [============================>.] - ETA: 1s - loss: 0.2097 - acc: 0.9389

KeyboardInterrupt: ignored

In [0]:
model = load_model (save_dir+save_file+'_acc_2.h5')

In [0]:
verify_datagen = ImageDataGenerator(rescale=1./255)
verify_generator = verify_datagen.flow_from_directory(src_valid,target_size=(h_size,w_size),shuffle=False)

verify_len = len(verify_generator.filenames)





val_length = len(valid_generator.filenames)
val_pred = model.predict_generator(verify_generator, verify_len, verbose=1 )



Found 1742 images belonging to 2 classes.
1742/1742 [==============================] - 786s 451ms/step


In [0]:
model.evaluate_generator(valid_generator,1742//32, verbose=1)

54/54 [==============================] - 832s 15s/step


[0.26562471933829696, 0.9397244547952467]

In [0]:
from  sklearn.metrics import roc_auc_score, matthews_corrcoef, accuracy_score
acc = accuracy_score(valid_generator.classes,val_pred)
auc  = acc = roc_auc_score(valid_generator.classes, val_pred)
mcorr  = acc = matthews_corrcoef(valid_generator.classes, val_pred)
print (acc, auc, mcorr)

ValueError: ignored

# Start Testing

In [31]:


test_datagen = ImageDataGenerator(
        rescale=1./255)




test_generator = test_datagen.flow_from_directory(
    directory=src_test,
    target_size=(h_size,w_size),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=False,
    seed=42
)


Found 20337 images belonging to 1 classes.


In [32]:
filenames = test_generator.filenames

print (filenames[:5])
nb_samples = len(filenames)
filenames.sort()

print (filenames[:5])

['test/signal_img_10000.png', 'test/signal_img_10001.png', 'test/signal_img_10002.png', 'test/signal_img_10003.png', 'test/signal_img_10004.png']
['test/signal_img_10000.png', 'test/signal_img_10001.png', 'test/signal_img_10002.png', 'test/signal_img_10003.png', 'test/signal_img_10004.png']


In [30]:
len(os.listdir(src_test+'test/'))

20337

In [42]:


filenames = test_generator.filenames

print (filenames[:5])
nb_samples = len(filenames)
filenames.sort()

print (filenames[:5])

print (nb_samples)

pred_list = []

for i in range(nb_samples//batch+1): 
  im_list = []
  
  for f in filenames[i*batch:i*batch + batch]:

    im = load_img(src_test+f)
    im = im.resize((288,144))
    img = img_to_array(im)
    img = img/255.
    im_list.append(img)
    test_batch = np.stack(im_list, axis=0)
    #print (test_batch.shape)
    
  pred = model.predict(test_batch)
  print (pred.shape, pred[:5])

  pred_list.append(pred)
    



['test/signal_img_10000.png', 'test/signal_img_10001.png', 'test/signal_img_10002.png', 'test/signal_img_10003.png', 'test/signal_img_10004.png']
['test/signal_img_10000.png', 'test/signal_img_10001.png', 'test/signal_img_10002.png', 'test/signal_img_10003.png', 'test/signal_img_10004.png']
20337
(64, 1) [[0.01384254]
 [0.0136404 ]
 [0.01340879]
 [0.01260435]
 [0.01226563]]
(64, 1) [[0.01144997]
 [0.01202736]
 [0.01012226]
 [0.01059752]
 [0.01168566]]
(64, 1) [[0.01393709]
 [0.01325278]
 [0.01283848]
 [0.0178371 ]
 [0.01731531]]
(64, 1) [[0.01166099]
 [0.01137304]
 [0.01234175]
 [0.01096598]
 [0.01128681]]
(64, 1) [[0.01139228]
 [0.01402469]
 [0.01441576]
 [0.01390938]
 [0.01214893]]
(64, 1) [[0.00944802]
 [0.00948472]
 [0.01000961]
 [0.01329372]
 [0.01299694]]
(64, 1) [[0.01394857]
 [0.01363147]
 [0.0135902 ]
 [0.01420554]
 [0.01321939]]
(64, 1) [[0.01173146]
 [0.01574819]
 [0.01608186]
 [0.01564784]
 [0.01117985]]
(64, 1) [[0.01046514]
 [0.01084487]
 [0.01060796]
 [0.01063655]
 [0.01

In [43]:
with open (src+'pred_list.pkl','wb') as handle:
  pickle.dump(pred_list, handle)

print ('pred_list.pkl saved')  

pred_list.pkl saved


In [44]:
final_pred = np.vstack(pred_list)
print (final_pred.shape)

final_pred = np.squeeze(final_pred)
print (final_pred.shape)

(20337, 1)
(20337,)


In [45]:
with open (src+'final_pred.pkl','wb') as handle:
  pickle.dump(final_pred, handle)

print ('final_pred.pkl saved')  

final_pred.pkl saved


In [0]:
import pickle 
with open(src+'test_pred.pkl','wb') as handle:
  pickle.dump(pred, handle)



In [47]:
with open(src+'filenames.pkl','wb') as handle:
  pickle.dump(filenames, handle)
print ('finenames saved')

finenames saved


In [0]:
test_df =pd.read_csv(src+'metadata_test.csv')

In [0]:
test_df.head()

,signal_id,id_measurement,phase
0,8712,2904,0
1,8713,2904,1
2,8714,2904,2
3,8715,2905,0
4,8716,2905,1


In [48]:
os.listdir(src)

['test.parquet',
 'sample_submission.csv',
 'metadata_test.csv',
 'metadata_train.csv',
 'train.parquet',
 'all.zip',
 'vsb_power_line_eda.ipynb',
 'train_test_data',
 'vsb_power_line_DenseNet121_1st__acc.h5',
 'vsb_power_line_DenseNet121_1st__loss.h5',
 'vsb_power_line_DenseNet121_1st__acc_2.h5',
 'vsb_power_line_DenseNet121_1st__loss_2.h5',
 'train_test_data_2',
 'id_label.csv',
 'new_train.csv',
 'pred_list.pkl',
 'final_pred.pkl',
 'filenames.pkl']

In [49]:
sample = pd.read_csv(src+'sample_submission.csv')
sample.head()

,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0


In [79]:
test_dict = {'test_id': filenames,
            'test_pred': final_pred}

test_df = pd.DataFrame(test_dict)

test_df.head()

,test_id,test_pred
0,test/signal_img_10000.png,0.013843
1,test/signal_img_10001.png,0.013640
2,test/signal_img_10002.png,0.013409
3,test/signal_img_10003.png,0.012604
4,test/signal_img_10004.png,0.012266


In [0]:
def get_id(f):
  return int(f[16:-4])

In [81]:
test_df['test_id'] = test_df['test_id'].map(get_id)
test_df=test_df.sort_values(by = ['test_id'])
test_df = test_df.reset_index()
test_df.head()

,index,test_id,test_pred
0,19049,8712,0.013165
1,19050,8713,0.012895
2,19051,8714,0.012301
3,19052,8715,0.012335
4,19053,8716,0.010915


In [84]:
test_df['test_pred'] = ( test_df['test_pred'] >0.4)*1
test_df.head()

,index,test_id,test_pred
0,19049,8712,0
1,19050,8713,0
2,19051,8714,0
3,19052,8715,0
4,19053,8716,0


In [85]:
sample['target'] = test_df['test_pred']
sample.head()

,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0


In [0]:
sample.to_csv(src+'powerline_1st_0102_01.csv',index= False)